In [ ]:
#import requests

#url = 'https://raw.githubusercontent.com/EinarOlafsson/spacr/main/code/spacr_data_generation.py'
#response = requests.get(url)
#with open('spacr_data_generation.py', 'w') as file:
#    file.write(response.text)
#    
#import spacr_data_generation as spacr

In [1]:
import sys
sys.path.append('/home/olafsson/Documents/repo/spacr/code')

import spacr_data_generation as spacr

Torch available: True
CUDA version: 11.8


In [ ]:
# 1. Organize images into npy stacks.
# 2. Preprocess images for object detection.
# 3. Generate masks.

settings = {'metadata_type':'cellvoyager',
            'experiment':'test',
            'channels':[0,1,2,3],
            'nucleus_channel':0,
            'nucleus_background':200,
            'nucleus_Signal_to_noise':10,
            'nucleus_CP_prob':0,
            'cell_channel':3,
            'cell_background':200,
            'cell_Signal_to_noise':10,
            'cell_CP_prob':-1,
            'pathogen_channel':2,
            'pathogen_background':200,
            'pathogen_Signal_to_noise':20,
            'pathogen_CP_prob':-1,
            'magnefication':20}

advanced_settings = {'preprocess':True,
                    'masks':True,
                    'custom_regex':None,
                    'save':True,
                    'plot':False,
                    'examples_to_plot':1,
                    'batch_size':100,
                    'randomize':True,
                    'timelapse':False,
                    'timelapse_displacement':None,
                    'timelapse_memory':3,
                    'timelapse_frame_limits':None,
                    'timelapse_remove_transient':False,
                    'timelapse_mode':'trackpy',
                     'timelapse_objects':['cells'],
                    'fps':2,
                    'remove_background':True,
                    'lower_quantile':0.01,
                    'merge':False,
                    'normalize_plots':True,
                    'all_to_mip':False,
                    'pick_slice':False,
                    'skip_mode':None,
                    'workers':30,
                    'verbose':True}

src = '/mnt/data/CellVoyager/40x/einar/tsg101HeLaToxoDsRed_20240225_113338/plate1'
spacr.preprocess_generate_masks(src,settings=settings,advanced_settings=advanced_settings)

========== settings ==========
source == /mnt/data/CellVoyager/40x/einar/tsg101HeLaToxoDsRed_20240225_113338/plate1
Bitdepth: uint16: cmap:inferno: figuresize:15
========== consolidating arrays ==========
regex == (?P<plateID>.*)_(?P<wellID>.*)_T(?P<timeID>.*)F(?P<fieldID>.*)L(?P<laserID>..)A(?P<AID>..)Z(?P<sliceID>.*)C(?P<chanID>.*).tif
========== creating single channel folders ==========
All_files:13832 in /mnt/data/CellVoyager/40x/einar/tsg101HeLaToxoDsRed_20240225_113338/plate1
Filename DC_DCAM#2_CAM2.tif did not match provided regex
Filename SC_BP525-50_40x W-Y1_M01_CH02.tif did not match provided regex
Filename DC_DCAM#3_CAM3.tif did not match provided regex
Filename SC_BP445-45_40x W-Y1_M01_CH01.tif did not match provided regex
Filename DC_DCAM#4_CAM4.tif did not match provided regex
Filename DC_DCAM#1_CAM1.tif did not match provided regex
Filename SC_BP676-29_40x W-Y1_M01_CH04.tif did not match provided regex
Filename SC_BP600-37_40x W-Y1_M01_CH03.tif did not match provided re

In [ ]:
# 1. Collect measurement data and save to database
# 2. Generate single object images.
#src = 'path'
src = '/mnt/data/CellVoyager/40x/einar/tsg101HeLaToxoDsRed_20240225_113338/plate1'

settings = {'input_folder':src+'/merged',
            'channels':[0,1,2,3],
            'cell_mask_dim':4,
            'cell_min_size':0,
            'nuclei_mask_dim':5,
            'nucleus_min_size':0,
            'pathogen_mask_dim':6,
            'pathogen_min_size':0,
            'cytoplasm_min_size':0}

advanced_settings = {'save_png':True,
                     'crop_mode':['cell'],
                     'use_bounding_box':True,
                     'png_size':[[224,224]],
                     'normalize':[2,99],
                     'png_dims':[0,1,2],
                     'normalize_by':'png', #'fov' or 'png'
                     'save_measurements':True,
                     'plot':False,
                     'plot_filtration':False,
                     'include_uninfected':False,
                     'dialate_pngs':False,
                     'dialate_png_ratios':[0.2],
                     'timelapse':False,
                     'timelapse_objects':'cell',
                     'max_workers':30}

annotation_settings = {'experiment':'test',
                      'cells':'HeLa',
                      'cell_loc':None,
                      'pathogens':['dsag1','dgra14_1','dgra14_2','dgra14_3'],
                      'pathogen_loc':[['c1','c2','c3','c13','c14','c15'],['c4','c5','c6','c16','c17','c18'],['c7','c8','c9','c19','c20','c21'],['c10','c11','c12','c22','c23','c24']],
                      'treatments':['cm','lovastatin_20uM'],
                      'treatment_loc':[['c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12'],['c13','c14','c15','c16','c17','c18','c19','c20','c21','c22','c23','c24']],
                      'channel_of_interest':1,
                      'compartments':['pathogen', 'cytoplasm'],
                      'measurement':'mean_intensity',
                      'nr_imgs':32,
                      'um_per_pixel':0.1}

if __name__ == "__main__":    
    spacr.measure_crop(settings, annotation_settings, advanced_settings)

In [ ]:
# Visualize masks and images
plot_settings = {'include_noninfected':False,
                 'include_multiinfected':True,
                 'include_multinucleated':True,
                 'filter_min_max':None,
                 'channel_dims':[0,1,2,3],
                 'remove_background':False,
                 'backgrounds':[100,100,100,100],
                 'cell_mask_dim':3,
                 'nucleus_mask_dim':0,
                 'pathogen_mask_dim':2,
                 'overlay':False,
                 'overlay_chans':[0,2,3],
                 'outline_thickness':2,
                 'outline_color':'gbr',
                 'normalization_percentiles':[1,99],
                 'normalize':True,
                 'print_object_number':True,
                 'nr':1,
                 'figuresize':20,
                 'cmap':'inferno',
                 'verbose':False}

src = 'path'
spacr.plot_merged(src, plot_settings)

In [ ]:
# Annotate single object images.

src = 'path'
spacr.annotate(db=src+'/measurements/measurements.db',
         image_type="cell_png", 
         channels=['r','g','b'],
         annotation_column='test',
         geom="3200x2000", 
         img_size=(200, 200), 
         rows=10, 
         columns=18)

In [ ]:
# Calculate recruitment.

metadata_settings = {'target':'protein',
                     'cell_types':['HeLa'],
                     'cell_plate_metadata':None,
                     'pathogen_types':['geneotype_1', 'geneotype_2', 'geneotype_3', 'geneotype_4'],
                     'pathogen_plate_metadata':[['c1', 'c7'],['c3', 'c8'],['c3','c4','c9','c10'],['c5','c6','c11','c12']],
                     'treatments':['cm', 'inhibitor'],
                     'treatment_plate_metadata':[['c1','c2','c3','c4','c5','c6'],['c7','c8','c9','c10','c11','c12']],
                     'metadata_types':'col',
                     'channel_dims':[0,1,2,3],
                     'cell_chann_dim':3,
                     'cell_mask_dim':4,
                     'nucleus_chann_dim':0,
                     'nucleus_mask_dim':5,
                     'pathogen_chann_dim':2,
                     'pathogen_mask_dim':6,
                     'channel_of_interest':1}

advanced_settings = {'plot':False,
                     'plot_nr':1,
                     'plot_control':True,
                     'figuresize':20,
                     'remove_background':True,
                     'backgrounds':100,
                     'include_noninfected':False,
                     'include_multiinfected':True,
                     'include_multinucleated':True,
                     'cells_per_well':10,
                     'pathogen_size_range':None,
                     'nucleus_size_range':None,
                     'cell_size_range':None,
                     'pathogen_intensity_range':None,
                     'nucleus_intensity_range':None,
                     'cell_intensity_range':None,
                     'target_intensity_min':100}

src = 'path'
dfs = spacr.analyze_recruitment(src,metadata_settings,advanced_settings)